## Sentiment to Strategy: Leveraging Forums Discussions to Guide Automated Trading Decisions 

---

### 0. Importing libraries

In [200]:
# Import necessary libraries
from pprint import pprint
import pandas as pd
import pandas as pd, json, urllib.request, io, pathlib
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
import praw
import prawcore
import csv
import time
from datetime import datetime, date, timezone
from dateutil.rrule import rrule, DAILY
from tqdm import tqdm
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import string
import spacy, pandas as pd, re, json
from spacy.pipeline import EntityRuler
from spacy.matcher import PhraseMatcher

---

### 1. Reddit API Data Extraction and Manipulation

In [201]:
# Initialize PRAW (Python Reddit API Wrapper)
user_agent = "StockMarket Sentiment Analysis"
reddit = praw.Reddit(
    client_id="SDD6BO5WvswKQ2DOZrioiQ",
    client_secret="BoGTdq9xg1Wu4kjy3zj_WBLbTInQvQ",
    user_agent=user_agent
)

# Initialize the subreddit
subreddit = reddit.subreddit("wallstreetbets")

In [202]:
# List to hold data for each post
posts = []

# Fetch posts from the subreddit
for submission in subreddit.new(limit=None):
    post_date = datetime.fromtimestamp(submission.created_utc)
    posts.append({
        'id': submission.id,
        'title': submission.title,
        'selftext': submission.selftext,
        'score': submission.score,
        'upvote_ratio': submission.upvote_ratio,       
        'created_date': post_date,
        'permalink': f"https://redd.it/{submission.id}"
    })

# Convert the list to a DataFrame
raw_df = pd.DataFrame(posts)
raw_df

,id,title,selftext,score,upvote_ratio,created_date,permalink
0,1ktui4f,spy puts go in the money,"happy hunting, lads",5,0.86,2025-05-23 23:02:44,https://redd.it/1ktui4f
1,1kttv6r,Red Tuesday,,10,0.92,2025-05-23 22:35:10,https://redd.it/1kttv6r
2,1kttf25,98% of gamblers quit before their big win,Withdrew $175K which is why all time looks off...,86,0.93,2025-05-23 22:15:53,https://redd.it/1kttf25
3,1ktt5kf,Trump greenlights Nippon merger with US Steel,https://www.cnbc.com/2025/05/23/trump-greenlig...,354,0.93,2025-05-23 22:04:49,https://redd.it/1ktt5kf
4,1ktszet,Weekend Discussion Thread for the Weekend of M...,This post contains content not supported on ol...,33,0.91,2025-05-23 21:57:42,https://redd.it/1ktszet
...,...,...,...,...,...,...,...
859,1kd2ui7,"Decided that 132% gain was good enough, and to...","Still have 1,548 shares that I'm holding for t...",321,0.96,2025-05-02 16:49:52,https://redd.it/1kd2ui7
860,1kd2n22,god i love volatility pt 3,playing mostly SPY swings and tech earnings. w...,30,0.89,2025-05-02 16:41:10,https://redd.it/1kd2n22
861,1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H...",Closed 40 $114c Reddit calls expiring May 2 fo...,44,0.89,2025-05-02 16:38:56,https://redd.it/1kd2l3k
862,1kd1usy,pltr calls printing,,40,0.85,2025-05-02 16:08:03,https://redd.it/1kd1usy


In [203]:
# Create a csv file with the data
if os.path.exists('wsb_posts.csv'):
    existing_df = pd.read_csv('wsb_posts.csv')
    combined_df = pd.concat([existing_df, raw_df]).drop_duplicates(subset=['id'])
    combined_df.to_csv('wsb_posts.csv', index=False)
else:
    raw_df.to_csv('wsb_posts.csv', index=False)

In [204]:
# Load the data from the csv file
df = pd.read_csv('wsb_posts.csv')
df

,id,title,selftext,score,upvote_ratio,created_date,permalink,text
0,1kqotlw,Whats your opinion about Rocket Lab?,I see many ppl hyping rocket lab and also boug...,1,1.00,2025-05-20 00:12:16,https://redd.it/1kqotlw,Whats your opinion about Rocket Lab? I see man...
1,1kqoadm,Fed's Powell has sounded the alarm for years a...,NaN,125,0.94,2025-05-19 23:49:38,https://redd.it/1kqoadm,Fed's Powell has sounded the alarm for years a...
2,1kqo5u3,Big beautiful bill,Am I screwed with my sp500 ETFs as a French in...,0,0.25,2025-05-19 23:44:20,https://redd.it/1kqo5u3,Big beautiful bill Am I screwed with my sp500 ...
3,1kqo1xz,Question on UNH Stock,"Should I sell UNH calls, covered calls or hold...",11,0.87,2025-05-19 23:39:50,https://redd.it/1kqo1xz,"Question on UNH Stock Should I sell UNH calls,..."
4,1kqmxov,Is this gonna print tomorrow,"Held these over the weekend, at its peak today...",0,0.44,2025-05-19 22:54:47,https://redd.it/1kqmxov,Is this gonna print tomorrow Held these over t...
...,...,...,...,...,...,...,...,...
1114,1ktui4f,spy puts go in the money,"happy hunting, lads",5,1.00,2025-05-23 23:02:44,https://redd.it/1ktui4f,NaN
1115,1ktubwa,$WULF FULL PORT YOLO,Full port yolo on $WULF. Mostly leaps I’ve bou...,3,0.80,2025-05-23 22:55:31,https://redd.it/1ktubwa,NaN
1116,1ktu6bz,Green Tuesday,Cause fk that other guy,1,0.60,2025-05-23 22:48:49,https://redd.it/1ktu6bz,NaN
1117,1ktu3i3,voicing disappointment with unsuccessful round...,NaN,14,0.82,2025-05-23 22:45:22,https://redd.it/1ktu3i3,NaN


In [205]:
# Read the S&P 500 companies list from the CSV file
csv_file = 'stocks.csv'
stocks_df = pd.read_csv(csv_file)

# Extract the ticker symbols (the column is usually named "Symbol")
tickers = stocks_df['Symbol'].tolist()

# Extract the names of the companies
companies = stocks_df['Name'].tolist()

In [206]:
# From company names remove the stock part and everything after it
def remove_stock_part(company_name):
    # Remove common stock suffixes (suffix + any following text)
    stock_suffixes = [' Inc.', ' Corp.', ' Ltd.', ' PLC', ' Co.', ' Class']
    for suffix in stock_suffixes:
        if suffix in company_name:
            return company_name.split(suffix)[0].strip()
    return company_name

# Apply the function to the companies list
companies = [remove_stock_part(company) for company in companies]

In [207]:
# Combine title and selftext into one column to search both
raw_df['text'] = raw_df['title'].fillna('') + " " + raw_df['selftext'].fillna('')

# Filter only the needed columns
filtered_reddit = raw_df.copy()
filtered_reddit.drop(columns=['title', 'selftext', 'score', 'upvote_ratio'], axis=1, inplace=True)
filtered_reddit

,id,created_date,permalink,text
0,1ktui4f,2025-05-23 23:02:44,https://redd.it/1ktui4f,"spy puts go in the money happy hunting, lads"
1,1kttv6r,2025-05-23 22:35:10,https://redd.it/1kttv6r,Red Tuesday
2,1kttf25,2025-05-23 22:15:53,https://redd.it/1kttf25,98% of gamblers quit before their big win With...
3,1ktt5kf,2025-05-23 22:04:49,https://redd.it/1ktt5kf,Trump greenlights Nippon merger with US Steel ...
4,1ktszet,2025-05-23 21:57:42,https://redd.it/1ktszet,Weekend Discussion Thread for the Weekend of M...
...,...,...,...,...
859,1kd2ui7,2025-05-02 16:49:52,https://redd.it/1kd2ui7,"Decided that 132% gain was good enough, and to..."
860,1kd2n22,2025-05-02 16:41:10,https://redd.it/1kd2n22,god i love volatility pt 3 playing mostly SPY ...
861,1kd2l3k,2025-05-02 16:38:56,https://redd.it/1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H..."
862,1kd1usy,2025-05-02 16:08:03,https://redd.it/1kd1usy,pltr calls printing


In [208]:
# Find matching tickers in the text
def matching_tickers(text):
    matching_tickers = [str(ticker) for ticker in tickers if str(ticker) in text]
    matching_tickers += [str(company) for company in companies if str(company).lower() in text.lower()]
    return matching_tickers

filtered_reddit['tickers'] = filtered_reddit['text'].apply(matching_tickers)
filtered_reddit

,id,created_date,permalink,text,tickers
0,1ktui4f,2025-05-23 23:02:44,https://redd.it/1ktui4f,"spy puts go in the money happy hunting, lads",[]
1,1kttv6r,2025-05-23 22:35:10,https://redd.it/1kttv6r,Red Tuesday,"[R, T]"
2,1kttf25,2025-05-23 22:15:53,https://redd.it/1kttf25,98% of gamblers quit before their big win With...,"[B, K, O, W]"
3,1ktt5kf,2025-05-23 22:04:49,https://redd.it/1ktt5kf,Trump greenlights Nippon merger with US Steel ...,"[B, J, S, T, U, ATI, ITT, nLIGHT, NN, Block]"
4,1ktszet,2025-05-23 21:57:42,https://redd.it/1ktszet,Weekend Discussion Thread for the Weekend of M...,"[C, D, M, R, T, W, Reddit]"
...,...,...,...,...,...
859,1kd2ui7,2025-05-02 16:49:52,https://redd.it/1kd2ui7,"Decided that 132% gain was good enough, and to...","[D, S]"
860,1kd2n22,2025-05-02 16:41:10,https://redd.it/1kd2n22,god i love volatility pt 3 playing mostly SPY ...,"[S, ATI]"
861,1kd2l3k,2025-05-02 16:38:56,https://redd.it/1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H...","[A, C, D, DD, DDT, DT, F, H, J, L, M, R, RDDT,..."
862,1kd1usy,2025-05-02 16:08:03,https://redd.it/1kd1usy,pltr calls printing,[]


In [209]:
# Remove substring tickers
def remove_substring_tickers(tickers):
    # Remove duplicates
    tickers = list(set(tickers))
    # Sort tickers by length in descending order
    tickers.sort(key=len, reverse=True)
    # Create a new list to store the filtered tickers
    filtered_tickers = []
    for ticker in tickers:
        # Check if the ticker is a substring of any other ticker in the list
        if not any(ticker in other_ticker for other_ticker in tickers if ticker != other_ticker):
            filtered_tickers.append(ticker)
    return filtered_tickers

# Apply the function to the filtered_reddit['tickers'] column
filtered_reddit['tickers'] = filtered_reddit['tickers'].apply(remove_substring_tickers)
filtered_reddit

,id,created_date,permalink,text,tickers
0,1ktui4f,2025-05-23 23:02:44,https://redd.it/1ktui4f,"spy puts go in the money happy hunting, lads",[]
1,1kttv6r,2025-05-23 22:35:10,https://redd.it/1kttv6r,Red Tuesday,"[T, R]"
2,1kttf25,2025-05-23 22:15:53,https://redd.it/1kttf25,98% of gamblers quit before their big win With...,"[B, O, W, K]"
3,1ktt5kf,2025-05-23 22:04:49,https://redd.it/1ktt5kf,Trump greenlights Nippon merger with US Steel ...,"[nLIGHT, Block, ATI, ITT, NN, J, S, U]"
4,1ktszet,2025-05-23 21:57:42,https://redd.it/1ktszet,Weekend Discussion Thread for the Weekend of M...,"[Reddit, M, D, T, W, C]"
...,...,...,...,...,...
859,1kd2ui7,2025-05-02 16:49:52,https://redd.it/1kd2ui7,"Decided that 132% gain was good enough, and to...","[S, D]"
860,1kd2n22,2025-05-02 16:41:10,https://redd.it/1kd2n22,god i love volatility pt 3 playing mostly SPY ...,"[ATI, S]"
861,1kd2l3k,2025-05-02 16:38:56,https://redd.it/1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H...","[Reddit, RDDT, XP, A, M, L, F, H, C, J]"
862,1kd1usy,2025-05-02 16:08:03,https://redd.it/1kd1usy,pltr calls printing,[]


In [210]:
# Remove ambiguous tickers
ambiguous_tickers = (
    list(string.ascii_uppercase)  # single-letter symbols
    + [
        # two-letter everyday words / abbreviations
        "AI", "EV", "IT", "ON", "OR", "NO", "OK", "GO", "SO", "DO", "UP", "EU", "US", "IP", 
        # three- to four-letter common words
        "ALL", "FUN", "BIG", "RUN", "NEW", "TOP", "YOU", "CAT", "DOG", "CAR", "SUN", "USA", 
        "IPO", "CAN", "NOW", "AND", "FOR", "OUT", "BUY", "WIN", "LOW", "HIGH", 
        "ONE", "TWO", "THREE", "FOUR", "FIVE", "SIX", "SEVEN", "EIGHT", "NINE", "TEN",
        # tech buzz-terms
        "CPU", "GPU", "VR", "AR", "NFT", "ML",
        # meme & slang tickers
        "YOLO", "HODL", "MOON", "LMAO", "LOL", "FOMO", "FUD", "WTF",
        # other 
        "Reddit", "WSB", "nan", "NAN"
    ])

# Remove ambiguous tickers from the filtered_reddit['tickers'] column
def remove_ambiguous_tickers(tickers):
    # Remove duplicates
    tickers = list(set(tickers))
    # Filter out ambiguous tickers
    tickers = [ticker for ticker in tickers if ticker not in ambiguous_tickers]
    return tickers

# Apply the function to the filtered_reddit['tickers'] column
filtered_reddit['tickers'] = filtered_reddit['tickers'].apply(remove_ambiguous_tickers)
filtered_reddit

,id,created_date,permalink,text,tickers
0,1ktui4f,2025-05-23 23:02:44,https://redd.it/1ktui4f,"spy puts go in the money happy hunting, lads",[]
1,1kttv6r,2025-05-23 22:35:10,https://redd.it/1kttv6r,Red Tuesday,[]
2,1kttf25,2025-05-23 22:15:53,https://redd.it/1kttf25,98% of gamblers quit before their big win With...,[]
3,1ktt5kf,2025-05-23 22:04:49,https://redd.it/1ktt5kf,Trump greenlights Nippon merger with US Steel ...,"[NN, ATI, ITT, nLIGHT, Block]"
4,1ktszet,2025-05-23 21:57:42,https://redd.it/1ktszet,Weekend Discussion Thread for the Weekend of M...,[]
...,...,...,...,...,...
859,1kd2ui7,2025-05-02 16:49:52,https://redd.it/1kd2ui7,"Decided that 132% gain was good enough, and to...",[]
860,1kd2n22,2025-05-02 16:41:10,https://redd.it/1kd2n22,god i love volatility pt 3 playing mostly SPY ...,[ATI]
861,1kd2l3k,2025-05-02 16:38:56,https://redd.it/1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H...","[XP, RDDT]"
862,1kd1usy,2025-05-02 16:08:03,https://redd.it/1kd1usy,pltr calls printing,[]


In [211]:
def keep_standalone_tickers(row):
    sentence = row["text"]
    cleaned = []
    for tkr in row["tickers"]:
        # look for the ticker as an independent token, optionally prefixed with $
        # (?<![A-Za-z0-9])  → left boundary is NOT a letter/number
        # (?![A-Za-z0-9])   → right boundary is NOT a letter/number
        pattern = rf'(?<![A-Za-z0-9])\$?{re.escape(tkr)}(?![A-Za-z0-9])'
        if re.search(pattern, sentence):
            cleaned.append(tkr)
    return cleaned

filtered_reddit["tickers"] = filtered_reddit.apply(keep_standalone_tickers, axis=1)
filtered_reddit

,id,created_date,permalink,text,tickers
0,1ktui4f,2025-05-23 23:02:44,https://redd.it/1ktui4f,"spy puts go in the money happy hunting, lads",[]
1,1kttv6r,2025-05-23 22:35:10,https://redd.it/1kttv6r,Red Tuesday,[]
2,1kttf25,2025-05-23 22:15:53,https://redd.it/1kttf25,98% of gamblers quit before their big win With...,[]
3,1ktt5kf,2025-05-23 22:04:49,https://redd.it/1ktt5kf,Trump greenlights Nippon merger with US Steel ...,[]
4,1ktszet,2025-05-23 21:57:42,https://redd.it/1ktszet,Weekend Discussion Thread for the Weekend of M...,[]
...,...,...,...,...,...
859,1kd2ui7,2025-05-02 16:49:52,https://redd.it/1kd2ui7,"Decided that 132% gain was good enough, and to...",[]
860,1kd2n22,2025-05-02 16:41:10,https://redd.it/1kd2n22,god i love volatility pt 3 playing mostly SPY ...,[]
861,1kd2l3k,2025-05-02 16:38:56,https://redd.it/1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H...",[RDDT]
862,1kd1usy,2025-05-02 16:08:03,https://redd.it/1kd1usy,pltr calls printing,[]


In [212]:
# Remove rows with empty 'tickers' list
filtered_reddit = filtered_reddit[filtered_reddit['tickers'].str.len() > 0]
filtered_reddit

,id,created_date,permalink,text,tickers
5,1ktskpw,2025-05-23 21:39:59,https://redd.it/1ktskpw,Informatica soars 23% as Salesforce reportedly...,"[Salesforce, INFA, Informatica, CRM]"
8,1ktrcyt,2025-05-23 20:48:59,https://redd.it/1ktrcyt,Next play - all in on draftkings $DKNG There’s...,[DKNG]
10,1ktr19o,2025-05-23 20:35:35,https://redd.it/1ktr19o,💀 YOLO’d SMCI Puts @ $0.70 → Cashed Out @ $1.6...,[SMCI]
15,1ktq15j,2025-05-23 19:54:43,https://redd.it/1ktq15j,Venture Global (VG) Set to Become the Largest ...,"[Venture Global, IMO, LNG, VG]"
16,1ktq10r,2025-05-23 19:54:33,https://redd.it/1ktq10r,Let's try like this - I bought today AES and P...,"[Pfizer, AES, RSI, Harmonic, PFE]"
...,...,...,...,...,...
854,1kd4ksx,2025-05-02 18:00:11,https://redd.it/1kd4ksx,"First time posting here, 10k left. All In on R...",[RDDT]
856,1kd49d3,2025-05-02 17:47:09,https://redd.it/1kd49d3,"$1900 to $45,000 in 3 months. First off, big t...","[META, Tesla]"
858,1kd3go4,2025-05-02 17:14:36,https://redd.it/1kd3go4,"$43K Profit – My “Triple Call Stack” on SPY, N...","[TSLA, NVDA]"
861,1kd2l3k,2025-05-02 16:38:56,https://redd.it/1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H...",[RDDT]


---

### 2. Sentiment Analysis

In [213]:
device = 0 if torch.cuda.is_available() else -1      # -1 ⇒ CPU
model_id = "ProsusAI/finBERT"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForSequenceClassification.from_pretrained(
    model_id,
    torch_dtype=torch.float16 if device==0 else torch.float32
)

sent_pipe1 = pipeline(
    task="sentiment-analysis",
    model=model,
    tokenizer=tokenizer,
    device=device,
    return_all_scores=True,    # **preferred** to top_k=None
    truncation=True
)

Device set to use cuda:0
C:\Users\andry\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\transformers\pipelines\text_classification.py:106: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


In [214]:
LABEL_TO_POLARITY = {"positive": +1, "neutral": 0, "negative": -1}

def finbert_score(text: str) -> float:
    scores = sent_pipe1(text, batch_size=1)[0]           # list[dict]
    # Convert to dict {label: prob}
    scores = {d["label"].lower(): d["score"] for d in scores}

    # Simple +/− difference (prob_pos − prob_neg)
    return scores["positive"] - scores["negative"]

filtered_reddit["sentiment"] = filtered_reddit["text"].apply(finbert_score)
filtered_reddit

C:\Users\andry\AppData\Local\Temp\ipykernel_8372\614457924.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_reddit["sentiment"] = filtered_reddit["text"].apply(finbert_score)


,id,created_date,permalink,text,tickers,sentiment
5,1ktskpw,2025-05-23 21:39:59,https://redd.it/1ktskpw,Informatica soars 23% as Salesforce reportedly...,"[Salesforce, INFA, Informatica, CRM]",-0.736918
8,1ktrcyt,2025-05-23 20:48:59,https://redd.it/1ktrcyt,Next play - all in on draftkings $DKNG There’s...,[DKNG],-0.196646
10,1ktr19o,2025-05-23 20:35:35,https://redd.it/1ktr19o,💀 YOLO’d SMCI Puts @ $0.70 → Cashed Out @ $1.6...,[SMCI],-0.014380
15,1ktq15j,2025-05-23 19:54:43,https://redd.it/1ktq15j,Venture Global (VG) Set to Become the Largest ...,"[Venture Global, IMO, LNG, VG]",-0.067108
16,1ktq10r,2025-05-23 19:54:33,https://redd.it/1ktq10r,Let's try like this - I bought today AES and P...,"[Pfizer, AES, RSI, Harmonic, PFE]",0.890249
...,...,...,...,...,...,...
854,1kd4ksx,2025-05-02 18:00:11,https://redd.it/1kd4ksx,"First time posting here, 10k left. All In on R...",[RDDT],0.210093
856,1kd49d3,2025-05-02 17:47:09,https://redd.it/1kd49d3,"$1900 to $45,000 in 3 months. First off, big t...","[META, Tesla]",0.122999
858,1kd3go4,2025-05-02 17:14:36,https://redd.it/1kd3go4,"$43K Profit – My “Triple Call Stack” on SPY, N...","[TSLA, NVDA]",0.142127
861,1kd2l3k,2025-05-02 16:38:56,https://redd.it/1kd2l3k,"$16,837 — Thanks for the RDDT Dip, You Paper-H...",[RDDT],0.208972


In [215]:
# make a dictionary that has the keys as the creation date of the post and the values as a dictionary of the tickers and their corresponding sentiment score summed up on that date
def create_sentiment_dict(df):
    sentiment_dict = {}
    for i, row in df.iterrows():
        date = row['created_date'].date()
        tickers = row['tickers']
        sentiment = row['sentiment']
        
        if date not in sentiment_dict:
            sentiment_dict[date] = {}
        
        for ticker in tickers:
            if ticker not in sentiment_dict[date]:
                sentiment_dict[date][ticker] = 0
            sentiment_dict[date][ticker] += sentiment
    
    return sentiment_dict

sentiment_dict = create_sentiment_dict(filtered_reddit)
sentiment_dict

{datetime.date(2025, 5, 23): {'Salesforce': -0.7369182333350182,
  'INFA': -0.7369182333350182,
  'Informatica': -0.7369182333350182,
  'CRM': -0.7369182333350182,
  'DKNG': -0.19664626568555832,
  'SMCI': -0.014379844069480896,
  'Venture Global': -0.06710822880268097,
  'IMO': -0.06710822880268097,
  'LNG': -0.06710822880268097,
  'VG': -0.06710822880268097,
  'Pfizer': 0.8902494953945279,
  'AES': 0.8902494953945279,
  'RSI': 0.8902494953945279,
  'Harmonic': 0.8902494953945279,
  'PFE': 0.8902494953945279,
  'AAPL': 0.4636433869600296,
  'GOOGL': 0.2755922246724367,
  'TSLA': 0.05564112588763237,
  'UNH': 0.38799571990966797,
  'MVST': 0.01325051300227642,
  'OKLO': 0.03253263980150223,
  'Tesla': 0.0019358638674020767,
  'LCID': 0.2243385650217533,
  'AMC': 0.2243385650217533,
  'NVDA': 0.1536855585873127,
  'META': 0.1536855585873127,
  'QBTS': -0.5453298389911652,
  'WLKP': 0.06674850359559059},
 datetime.date(2025, 5, 22): {'Intuit': 0.9122290443629026,
  'IONQ': 1.008111972361

In [216]:
# Convert sentiment_dict into a DataFrame and fill missing values with 0
sentiment_df = pd.DataFrame.from_dict(sentiment_dict, orient='index').fillna(0)

# Ensure the index is sorted (dates in ascending order)
sentiment_df.sort_index(inplace=True)
sentiment_df

,Salesforce,INFA,Informatica,CRM,DKNG,SMCI,Venture Global,IMO,LNG,VG,...,Commvault Systems,NET,SU,SOFI,CC,PB,QQQX,TSM,JD,AIP
2025-05-02,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.880135,0.545319,0.538456
2025-05-03,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-0.262786,0.049127,0.049127,0.000000,0.000000,0.000000
2025-05-04,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-05-05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.896298,0.085657,0.240592,-0.052041,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-05-06,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-05-07,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-05-08,0.113937,0.000000,0.000000,0.113937,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-05-09,0.925770,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-05-10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2025-05-11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


---

### 3. Stock Market Comparison

---

### 4. Visualization 